In [1]:
from __future__ import print_function

import sys
import threading

from grpc.beta import implementations
import numpy
import tensorflow as tf
from datetime import datetime

from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2

In [2]:
!pip install tensorflow_serving_api==1.13.0


You are using pip version 19.0.1, however version 19.2.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
!pip show tensorflow_serving_api

Name: tensorflow-serving-api
Version: 1.13.0
Summary: TensorFlow Serving Python API.
Home-page: http://tensorflow.org/serving
Author: Google Inc.
Author-email: tensorflow-serving-dev@googlegroups.com
License: Apache 2.0
Location: /opt/conda/lib/python3.6/site-packages
Requires: protobuf, grpcio, tensorflow
Required-by: 


In [4]:
from __future__ import print_function

import sys
import threading

from grpc.beta import implementations
import numpy
import tensorflow as tf
from datetime import datetime

from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2

In [5]:
tf.app.flags.DEFINE_string("server", "10.128.0.46:9000", "PredictionService host:port")
FLAGS = tf.app.flags.FLAGS

In [6]:
def do_inference(hostport):
    """Tests PredictionService with concurrent requests.
  Args:
    hostport: Host:port address of the PredictionService.
  Returns:
    pred values, ground truth labels, processing time
  """
    # connect to server
    host, port = hostport.split(":")
    channel = implementations.insecure_channel(host, int(port))
    stub = prediction_service_pb2.beta_create_PredictionService_stub(channel)

    # prepare request object
    request = predict_pb2.PredictRequest()
    request.model_spec.name = "default"
    request.model_spec.signature_name = "prediction"

    # Randomly generate some test data
    temp_data = numpy.random.randn(100, 3).astype(numpy.float32)
    data, label = (
        temp_data,
        numpy.sum(temp_data * numpy.array([1, 2, 3]).astype(numpy.float32), 1),
    )
    request.inputs["input"].CopyFrom(
        tf.contrib.util.make_tensor_proto(data, shape=data.shape)
    )

    # make inference and clock the time
    now = datetime.now()
    result = stub.Predict(request, 5.0)  # 5 seconds
    waiting = datetime.now() - now
    return result, label, waiting.microseconds


In [7]:
def main(_):

    if not FLAGS.server:
        print("please specify server host:port")
        return

    result, label, waiting = do_inference(FLAGS.server)
    print("Result is: ", result)
    print("Actual label is: ", label)
    print("Waiting time is: ", waiting, "microseconds.")

In [8]:
tf.app.run()

/opt/conda/lib/python3.6/site-packages/tensorflow_serving/apis/prediction_service_pb2.py:131: DeprecationWarning: beta_create_PredictionService_stub() method is deprecated. This method will be removed in near future versions of TF Serving. Please switch to GA gRPC API in prediction_service_pb2_grpc.
  'prediction_service_pb2_grpc.', DeprecationWarning)



For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Result is:  outputs {
  key: "output"
  value {
    dtype: DT_FLOAT
    tensor_shape {
      dim {
        size: 100
      }
      dim {
        size: 1
      }
    }
    float_val: 4.55661153793335
    float_val: 1.242213487625122
    float_val: 4.095940589904785
    float_val: 0.03343285992741585
    float_val: -0.3759818971157074
    float_val: -1.9288034439086914
    float_val: 3.333224296569824
    float_val: -1.4037739038467407
    float_val: 0.612320065498352
    float_val: 3.054539203643799
    float_val: 3.407688617706299
    float_val: 5.076531887054443
    float_val: -1.8865214586257935
    float_val: -1.6547049283981323
    float_val: -1.0727590322494507
    float_val: 6.138456344604492
    float_val: 1.8794984817504883
    float_val: -2.0373

/opt/conda/lib/python3.6/site-packages/tensorflow/contrib/lite/python/__init__.py:26: PendingDeprecationWarning: WARNING: TF Lite has moved from tf.contrib.lite to tf.lite. Please update your imports. This will be a breaking error in TensorFlow version 2.0.
  _warnings.warn(WARNING, PendingDeprecationWarning)


SystemExit: 

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3304: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
